# Notebook to generate VHDL for SAD

In [1]:
import numpy as np 
import math

In [43]:
################################################
################## Parametros ##################
block_size = 2
bit_width = 8
proc_parallel = True        # True: processamento paralelo de bloco todo, False: processamento por linha
files = ['adder.vhd']       # Lista de componentes para add
adder_name = 'adder'        # Nome do somador (só pra facilitar)

if proc_parallel:
    file_name = ('sad_' + str(int(math.sqrt(block_size))) + 'x' + str(int(math.sqrt(block_size))) + '_parallel')
else:
    file_name = ('sad_' + str(int(math.sqrt(block_size))) + 'x' + str(int(math.sqrt(block_size))) + '_line')

pack_name = 'type_pack'
rewrite_package = False
################################################

In [44]:
################################################# Basis #################################################
if proc_parallel:                           # Ex: bloco 4x4 processa 16 em paralelo
    limit = block_size
else:
    limit = int(math.sqrt(block_size))      # Ex: bloco 4x4 processa 4 vezes cada linha de 4

################################################# Port ################################################
port = ('''ori: in input_'''+str(limit)+''';
    ref: in input_'''+str(limit)+''';
    sad_out: out std_logic_vector(''' + str(math.ceil(np.log2(block_size))+bit_width) + ''' downto 0)''')

################################################# Components ################################################
components = []
line_qnt = 0

for comp in files:
    with open(comp, "r") as file:
        lines = file.readlines()

    line_split = []
    for i in range(len(lines)):                # Separa os componentes de cada linha lida
        line_split.append(lines[i].split())

    end_line = 0
    for i in range(0,len(lines),1):                 # Itera em todas linhas        
        if line_split[i] != []:                     # Verifica se não é uma linha vazia
            if line_split[i][0] == 'entity':        # Vê se é entity                
                begin_line = i
            elif line_split[i][0] == 'end':         # Vê se é o final de entity
                if end_line == 0:
                    end_line = i

    for l in range(begin_line,end_line+1,1):        # Pega todas as linhas da entity
        components.append(lines[l])

    line1_split = components[line_qnt].split()      # Alterando a palavra 'entity' 
    line1_split[0] = 'component'                    # para 'component'
    line1_split.append('\n')
    components[line_qnt] = ' '.join(line1_split)

    linelast_split = components[-1].split()         # Alterando a última palavra 
    linelast_split[-1] = 'component;'               # para 'component'
    components[-1] = ' '.join(linelast_split)

    components.append('\n\n')
    line_qnt = len(components)

components_concat = ' '.join(components)         #  Junta todos componentes

################################################# Signals ################################################
negs = []   # Sinais negativos reference para entrar em somador e virar subtracao
for i in range(0,limit):
    negs.append('signal neg_ref_'+str(i)+': std_logic_vector('+str(bit_width-1)+' downto 0);')
negs.append('\n')
negs_concat = '\n'.join(negs)

sub = []    # Sinais da 1a subtracao (estrutura basica)
for i in range(0,limit):
    sub.append('signal sub_' + str(i) + ': std_logic_vector(' + str(bit_width) + ' downto 0);')
sub.append('\n')
sub_concat = '\n'.join(sub)

abs = []        # Sinais do calculo do absoluto (estrutura basica)
out_abs = []    # Lista com nome dos sinais que saem do absoluto
for i in range(0,limit):
    out_abs.append('abs_'+str(i))
    abs.append('signal abs_' + str(i) + ': std_logic_vector(' + str(bit_width) + ' downto 0);')
abs.append('\n')
abs_concat = '\n'.join(abs)

sum0 = []       # Sinais da 1a soma (estrutura basica)
out_sum = []    # Lista com nome dos sinais que saem da 1a soma
for i in range(0,limit,2):
    out_sum.append('sum_'+str(i) + str(i+1))
    sum0.append('signal sum_' + str(i) + str(i+1) + ': std_logic_vector(' + str(bit_width+1) + ' downto 0);')
sum0.append('\n')
sum0_concat = '\n'.join(sum0)

after_sums = []         # Sinais dos outros niveis de soma abaixo 
out_after_sums = []     # Lista com nome dos sinais que saem das outras somas
for k in range(1,math.ceil(np.log2(limit))):    # Qtd de niveis de soma
    qtd_sum = int(limit/(2**(k+1)))             # Qtd de somadores por nivel
    for j in range(0,qtd_sum):
        after_sums.append('signal sum_after_'+str(k)+str(j)+': std_logic_vector('+str(bit_width+k+1)+' downto 0);')
        out_after_sums.append('sum_after_'+str(k)+str(j))
after_sums.append('\n')
after_sums_concat = '\n'.join(after_sums)

signals = []
signals.append(negs_concat + sub_concat + abs_concat + sum0_concat + after_sums_concat)
signals_concat = ''.join(signals)
# print(signals_concat)

################################################# Assign ################################################
neg_assign = []     # Definindo os fios negativos pra entrar no somador (= subtrator)
for i in range(0,limit):
    neg_assign.append('neg_ref_'+str(i)+' <= std_logic_vector(resize(-signed(ref('+str(i)+')),'+str(bit_width)+'));')
neg_assign.append('\n')
neg_assign_concat = '\n'.join(neg_assign)

abs_assign = []     # Definindo os fios que saem do absoluto
for i in range(0,limit):
    abs_assign.append('abs_'+str(i)+' <= std_logic_vector(abs(signed(sub_'+str(i)+')));')
abs_assign.append('\n')
abs_assign_concat = '\n'.join(abs_assign)

assigns = []        # Juntando todos os assigns
assigns.append(neg_assign_concat + abs_assign_concat)
assigns_concat = ''.join(assigns)

print(out_sum)
print(out_after_sums)

['sum_01']
[]


In [45]:
################################################# Port maps #################################################
sub_map = []            #  Port map dos componentes subtratores
for i in range(0,limit):
    sub_map.append('SUB'+str(i)+': '+adder_name+' generic map ('+str(bit_width)+') port map (ori'+str(i)+',neg_ref_'+str(i)+',sub_'+str(i)+');')
sub_map.append('\n')
sub_map_concat = '\n'.join(sub_map)
print(sub_map_concat)

sums_map = []
count = 0
for i in range(0,limit,2):
    sums_map.append('ADD0'+str(count)+': '+adder_name+' generic map ('+str(bit_width+1)+') port map ('+out_abs[i]+','+out_abs[i+1]+','+out_sum[count]+');')
    count = count + 1
sums_map.append('\n')
sums_map_concat = '\n'.join(sums_map)
print(sums_map_concat)

after_sums_map = []     # Sinais dos outros niveis de soma abaixo 
n = 0 
m = 0
l = 0
count = 0
for k in range(1,math.ceil(np.log2(limit))+1):          # Qtd de niveis de soma         2
    qtd_sum_map = int(limit/(2**(k+1)))                 # Qtd de somadores por nivel    2, 1
    for j in range(0,qtd_sum_map):
        if k == 1:
            after_sums_map.append('ADD'+str(k)+str(j)+': '+adder_name+' generic map ('+str(bit_width+k+1)+') port map ('+out_sum[count]+','+out_sum[count+1]+','+out_after_sums[n]+');')
            count = count + 2
            n = n + 1
        elif k >= 2: # 8 somadores, 4, 2, 1
            after_sums_map.append('ADD'+str(k)+str(j)+': '+adder_name+' generic map ('+str(bit_width+k+1)+') port map ('+out_after_sums[m]+','+out_after_sums[m+1]+','+out_after_sums[l+qtd_sum_map*(2**(k-1))]+');')
            m = m + 2
            l = l + 1
            
after_sums_map.append('\n')
after_sums_map_concat = '\n'.join(after_sums_map)

print(after_sums_map_concat)



SUB0: adder generic map (8) port map (ori0,neg_ref_0,sub_0);
SUB1: adder generic map (8) port map (ori1,neg_ref_1,sub_1);


ADD00: adder generic map (9) port map (abs_0,abs_1,sum_01);






In [5]:
# Texto do VHDL
vhdl = ('''library ieee;
use ieee.std_logic_1164.all;
use ieee.std_logic_unsigned.all;
use ieee.numeric_std.all;
use work.''' + pack_name + '''.all;

entity ''' + file_name + ''' is 
port(
    ''' + port + '''
);
end ''' + file_name + ''';

architecture arch_sad of ''' + file_name + ''' is 

-- Components 
''' + components_concat + '''
-- Signals
''' + signals_concat + '''

begin'''
)

print(vhdl)

library ieee;
use ieee.std_logic_1164.all;
use ieee.std_logic_unsigned.all;
use ieee.numeric_std.all;
use work.type_pack.all;

entity sad_4x4_parallel is 
port(
    ori: in input_16;
    ref: in input_16;
    sad_out: out std_logic_vector(12 downto 0)
);
end sad_4x4_parallel;

architecture arch_sad of sad_4x4_parallel is 

-- Components 
component adder is 
 generic (N: integer);
 port(
 		A, B: in std_logic_vector(N-1 downto 0);
 		S: out std_logic_vector(N downto 0)
 );
 end component; 


-- Signals
signal neg_ref_0: std_logic_vector(7 downto 0);
signal neg_ref_1: std_logic_vector(7 downto 0);
signal neg_ref_2: std_logic_vector(7 downto 0);
signal neg_ref_3: std_logic_vector(7 downto 0);
signal neg_ref_4: std_logic_vector(7 downto 0);
signal neg_ref_5: std_logic_vector(7 downto 0);
signal neg_ref_6: std_logic_vector(7 downto 0);
signal neg_ref_7: std_logic_vector(7 downto 0);
signal neg_ref_8: std_logic_vector(7 downto 0);
signal neg_ref_9: std_logic_vector(7 downto 0);
signal neg_re

## Geração do Package

In [6]:
# Criando package de tipo
pack = ('''library ieee;
use ieee.std_logic_1164.all;
use ieee.std_logic_unsigned.all;
use ieee.numeric_std.all;

package ''' + pack_name + ''' is
    
    constant bit_width: natural := ''' + str(bit_width) + '''; 
    type input_line is array (0 to ''' + str(limit-1) + ") of std_logic_vector(" + str(bit_width-1) + ''' downto 0);

end ''' + pack_name + ';'
)

if rewrite_package:
    pack_file = open((pack_name + ".vhd"), "w")
    pack_file.write(pack)
    pack_file.close()